# Applied Data Science Capstone on Coursera

This notebook is used for the Coursera's Capstone Project course on Applied Data Science.


In [6]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course")

Hello Capstone Project Course


 # Clustering Toronto
 ## First Part: Scraping Toronto Neighborhoods
 
 First we install beautifulsoup and requests.

In [7]:
!conda install beautifulsoup4 --yes
!conda install requests --yes

Solving environment: done

## Package Plan ##

  environment location: /home/vininim/anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py37h06a4308_0         2.9 MB
    ------------------------------------------------------------
                                           Total:         2.9 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.9.2-py37h89c1867~ --> pkgs/main::conda-4.9.2-py37h06a4308_0



conda-4.9.2          | 2.9 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.



Then we scrape wikipedia page.

In [8]:
import requests
from bs4 import BeautifulSoup
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
soup = BeautifulSoup(r.text)
#Assuming the page stay the same, the first table is the one we want
table = soup.find_all('table')[0]
rows = table.find_all('tr')

df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
# skip first row
for i in range(1,len(rows)):
    tds = rows[i].find_all('td')
    code = tds[0].contents[0].strip()
    borough = tds[1].contents[0].strip()
    neighborhood = tds[2].contents[0].strip()
    
    if borough == "Not assigned":
        continue
    if neighborhood == "Not assigned":
        neighborhood = borough
    newrow = pd.DataFrame({'PostalCode': [code], 'Borough': [borough], 'Neighborhood': [neighborhood]})
    idx = df.loc[df.isin([code]).any(axis=1)].index.tolist()
    if idx:
        df.iloc(idx[0])['Neighborhood'] = df.iloc(idx[0])['Neighborhood'] + "," + neighborhood
    else:
        df = df.append(newrow)

print(df.shape)

(103, 3)


## Second Part: Getting coordinates for neighborhoods

In [9]:
!wget -O Geospatial_data.csv https://cocl.us/Geospatial_data


--2020-12-07 16:51:31--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.194, 169.63.96.176
Connecting to cocl.us (cocl.us)|169.63.96.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-12-07 16:51:33--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-12-07 16:51:33--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]


In [10]:
coord_df = pd.read_csv("Geospatial_data.csv")
coord_df.rename(columns={"Postal Code": "PostalCode"},inplace=True)
df_result = pd.merge(df,coord_df,on="PostalCode")
df_result.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Third part: getting venues in neighbourhoods

In [11]:
# @hidden_cell

CLIENT_ID = "ZRVP1PDJNJUWZRXHCJCN2C0SZYMDJYHZAYMOKXLZY4CZPKYU"
CLIENT_SECRET = "ZZYL0ZTQZF4CWEVFHFTZS2YLLN51AXHKP5VTIGO5MJA2AUFH"
VERSION = '20180605'


Use Foursquare API to get venues nearby neighborhoods. (need to use CLIENT_ID and CLIENT_SECRET)

In [22]:
LIMIT = 100 
radius = 500 
neigh_lat = df_result.loc[0,"Latitude"]
neigh_lon = df_result.loc[0,"Longitude"]
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_lon, 
    radius, 
    LIMIT)

result = requests.get(url).json()
from pandas import json_normalize
venues = json_normalize(result['response']['groups'][0]['items'])
venues = venues.loc[:,['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']]
venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.751976,-79.332140
1,Variety Store,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",43.751974,-79.333114


Get venue category.

In [23]:
def get_category(row):
    return row['venue.categories'][0]['name']
venues['venue.categories']=venues.apply(get_category,axis=1)
venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


Get venues for every neighborhood.

In [24]:
venues_list = []
for index, row in df_result.iterrows():
    neigh_lat = row["Latitude"]
    neigh_lon = row["Longitude"]
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neigh_lat, 
        neigh_lon, 
        radius, 
        LIMIT)
    result = requests.get(url).json()
    try:
        venues = json_normalize(result['response']['groups'][0]['items'])
    except:
        continue
    if venues.empty:
        continue
    venues = venues.loc[:,['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']]
    venues['venue.categories']=venues.apply(get_category,axis=1)
    for index2, row_venues in venues.iterrows():
        venues_list.append((row['Neighborhood'], row["Latitude"], row["Longitude"], row_venues['venue.name'], row_venues['venue.categories']))
venues_list[0:5]

[('Parkwoods', 43.7532586, -79.3296565, 'Brookbanks Park', 'Park'),
 ('Parkwoods', 43.7532586, -79.3296565, 'Variety Store', 'Food & Drink Shop'),
 ('Victoria Village',
  43.725882299999995,
  -79.31557159999998,
  'Victoria Village Arena',
  'Hockey Arena'),
 ('Victoria Village',
  43.725882299999995,
  -79.31557159999998,
  'Tim Hortons',
  'Coffee Shop'),
 ('Victoria Village',
  43.725882299999995,
  -79.31557159999998,
  'Portugril',
  'Portuguese Restaurant')]

Get one hot representation of categories per venue and group by neighborhood getting a mean.

In [25]:
venues_df = pd.DataFrame(venues_list)
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'Venue', 'Category']
venues_onehot = pd.get_dummies(venues_df[['Category']])
venues_onehot['Neighborhood'] = venues_df['Neighborhood']
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped.shape

(96, 274)

K-means clustering on mean frequency of category.

In [29]:
from sklearn.cluster import KMeans
kclusters=5
venues_clustering = venues_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters = kclusters)
kmeans.fit(venues_clustering)
kmeans.labels_[0:10]

array([0, 4, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Get labels on dataframe with long/lat.

In [30]:
venues_grouped['Cluster'] = kmeans.labels_
neighborhood_clustered = pd.merge(df_result,venues_grouped, on="Neighborhood")
columns = ['Neighborhood', 'Latitude', 'Longitude', 'Cluster']
neighborhood_clustered = neighborhood_clustered.loc[:,columns]
neighborhood_clustered.head(10)

,Neighborhood,Latitude,Longitude,Cluster
0,Parkwoods,43.753259,-79.329656,2
1,Victoria Village,43.725882,-79.315572,0
2,"Regent Park, Harbourfront",43.654260,-79.360636,0
3,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0
4,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0
5,"Malvern, Rouge",43.806686,-79.194353,3
6,Don Mills,43.745906,-79.352188,0
7,Don Mills,43.725900,-79.340923,0
8,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,0
9,"Garden District, Ryerson",43.657162,-79.378937,0


Draw neighborhoods on map.

In [19]:
!conda install -c conda-forge folium --yes

Solving environment: done

## Package Plan ##

  environment location: /home/vininim/anaconda3

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.11.0              |             py_0          61 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          61 KB

The following packages will be UPDATED:

  folium                                         0.5.0-py_0 --> 0.11.0-py_0

The following packages will be SUPERSEDED by a higher-priority channel:

  conda               pkgs/main::conda-4.9.2-py37h06a4308_0 --> conda-forge::conda-4.9.2-py37h89c1867_0



folium-0.11.0        | 61 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [31]:
toronto_latitude=43.6532
toronto_longitude=-79.3832
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

map_tor = folium.Map(location=[toronto_latitude,toronto_longitude],zoom_start=11)

colors_array = cm.rainbow(np.linspace(0,1,kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, long, name, cluster in zip(neighborhood_clustered['Latitude'],neighborhood_clustered['Longitude'],neighborhood_clustered['Neighborhood'], neighborhood_clustered['Cluster']):
    label = folium.Popup(str(name) + ' Cluster ' + str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,long],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7
    ).add_to(map_tor)
    
map_tor